### Load libraries

In [1]:
#General
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from math import radians, cos, sin, asin, sqrt
import time
import math
import matplotlib.pylab as plt
import datetime as dt
#Scikit
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.cluster import MiniBatchKMeans
#XGBoost
import xgboost as xgb
from xgboost.sklearn import XGBRegressor


C:\Users\Clement\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Loading data

In [2]:
dfTest = pd.read_csv("Data/test.csv")
dfTrain = pd.read_csv("Data/train.csv")

#OSRM Data
OSRMtest = pd.read_csv("Data/OSRMtest.csv")
OSRMtrain = pd.read_csv("Data/OSRMtrain.csv")

dfTrain["OSMR_distance"] = OSRMtrain["total_distance"]
dfTrain["OSMR_duration"] = OSRMtrain["total_duration"]

dfTest["OSMR_distance"] = OSRMtest["total_distance"]
dfTest["OSMR_duration"] = OSRMtest["total_duration"]

### Define functions needed

In [3]:
#Adding distance in km
# credit to: https://stackoverflow.com/questions/15736995/how-can-i-quickly-estimate-the-distance-between-two-latitude-longitude-points
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

### Feature engineering

In [4]:
#Concat data for featuring engineering
features = dfTest.columns
label = "trip_duration"
data = pd.concat([dfTrain[features], dfTest], keys=['train','test'])

#Clustering on location in NY
t0 = dt.datetime.now()
coords = np.vstack((data[['pickup_latitude', 'pickup_longitude']].values,
                    data[['dropoff_latitude', 'dropoff_longitude']].values))
sample_ind = np.random.permutation(len(coords))[:500000]
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coords[sample_ind])

data['pickup_cluster'] = kmeans.predict(data[['pickup_latitude', 'pickup_longitude']])
data['dropoff_cluster'] = kmeans.predict(data[['dropoff_latitude', 'dropoff_longitude']])
t1 = dt.datetime.now()
print('Clustering time : %i seconds' % (t1 - t0).seconds)

#Add datetime infos in diff columns
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['pickup_year'] = data['pickup_datetime'].dt.year
data['pickup_month'] = data['pickup_datetime'].dt.month
data['pickup_weekday'] = data['pickup_datetime'].dt.weekday
data['pickup_day'] = data['pickup_datetime'].dt.day
data['pickup_hour'] = data['pickup_datetime'].dt.hour
data['pickup_minute'] = data['pickup_datetime'].dt.minute

data.drop('pickup_datetime', axis=1, inplace=True)
data.drop('pickup_year', axis=1, inplace=True)

#Change storeflag in continue
data_dict = {'Y':1, 'N':0}
data_tf = data['store_and_fwd_flag'].map(data_dict)
data['store_and_fwd_flag'].update(data_tf)

#Add distance (takes a while)
t0 = dt.datetime.now()
data['distance'] = data.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
t1 = dt.datetime.now()
print('Computing distance : %i seconds' % (t1 - t0).seconds)

#Drop tails
train_set = data.loc['train']
test_set = data.loc['test']
data_temp = train_set
data_temp[label] = dfTrain[label]
data_temp = data_temp[data_temp[label] < 1800000]
target = data_temp[label]
target_log = np.log(target+1)
train_set = data_temp
data = train_set

#Drop id and labels
data = train_set.drop(['id', label], axis=1).astype(float)

Clustering time : 3 seconds
Computing distance : 156 seconds


### Train model XGBoost with optimal parameters

In [16]:
#Best params obtained with hypertunning below

model = XGBRegressor(
 learning_rate =0.1,
 n_estimators=10000,
 max_depth=7,
 min_child_weight=3,
 gamma=0.4,
 subsample=0.9,
 colsample_bytree=0.9,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 reg_alpha=100,
 seed=27)

XGBmodelfit(xgb1, data, target_log)

X_train: (1239844, 17)  Y_train: (1239844,)  X_test: (218796, 17)  Y_test: (218796,)
[0]	validation_0-rmse:5.42159
Will train until validation_0-rmse hasn't improved in 50 rounds.
[20]	validation_0-rmse:0.79084
[40]	validation_0-rmse:0.425122
[60]	validation_0-rmse:0.407884
[80]	validation_0-rmse:0.403298
[100]	validation_0-rmse:0.400656
[120]	validation_0-rmse:0.397921
[140]	validation_0-rmse:0.395894
[160]	validation_0-rmse:0.394543
[180]	validation_0-rmse:0.393253
[200]	validation_0-rmse:0.39211
[220]	validation_0-rmse:0.390937
[240]	validation_0-rmse:0.389834
[260]	validation_0-rmse:0.388906
[280]	validation_0-rmse:0.388168
[300]	validation_0-rmse:0.387335
[320]	validation_0-rmse:0.386684
[340]	validation_0-rmse:0.38609
[360]	validation_0-rmse:0.385463
[380]	validation_0-rmse:0.385048
[400]	validation_0-rmse:0.384609
[420]	validation_0-rmse:0.384177
[440]	validation_0-rmse:0.383732
[460]	validation_0-rmse:0.383446
[480]	validation_0-rmse:0.383087
[500]	validation_0-rmse:0.382727
[5

[4800]	validation_0-rmse:0.374544
[4820]	validation_0-rmse:0.374542
[4840]	validation_0-rmse:0.374532
[4860]	validation_0-rmse:0.374526
[4880]	validation_0-rmse:0.374521
[4900]	validation_0-rmse:0.374517
[4920]	validation_0-rmse:0.37451
[4940]	validation_0-rmse:0.374507
[4960]	validation_0-rmse:0.3745
[4980]	validation_0-rmse:0.374493
[5000]	validation_0-rmse:0.374487
[5020]	validation_0-rmse:0.374483
[5040]	validation_0-rmse:0.374482
[5060]	validation_0-rmse:0.37448
[5080]	validation_0-rmse:0.374476
[5100]	validation_0-rmse:0.374459
[5120]	validation_0-rmse:0.374454
[5140]	validation_0-rmse:0.374453
[5160]	validation_0-rmse:0.374456
[5180]	validation_0-rmse:0.374447
[5200]	validation_0-rmse:0.374444
[5220]	validation_0-rmse:0.374442
[5240]	validation_0-rmse:0.374439
[5260]	validation_0-rmse:0.374435
[5280]	validation_0-rmse:0.374431
[5300]	validation_0-rmse:0.374432
[5320]	validation_0-rmse:0.374426
[5340]	validation_0-rmse:0.374418
[5360]	validation_0-rmse:0.374408
[5380]	validation_

### Submission

In [17]:
dataSub = test_set.drop('id', axis=1).astype(float)
Y_eval_log = model.predict(dataSub)
Y_eval = np.exp(Y_eval_log.ravel())-1
submit_file = pd.DataFrame({'id': test_set['id'], 'trip_duration': Y_eval})
submit_file.to_csv('Data/submission.csv',index=False)

### Tunning hyperparameters for XGBoost

Define function that we are going to use.

In [12]:
def XGBmodelfit(alg, Xtrain, Ytrain):
    
    t0 = dt.datetime.now()
    
    #Get Parameters
    xgb_param = alg.get_xgb_params()
    
    #Define matrix
    xgtrain = xgb.DMatrix(Xtrain.values, label=Ytrain.values)
    
    #Define model for CV
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=2,
            metrics='rmse', early_stopping_rounds=50, verbose_eval=10)
    
    #Set optimal n_estimators to alg
    alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(Xtrain, Ytrain,eval_metric='rmse')
        
    #Predict training set:
    dtrain_predictions = alg.predict(Xtrain)
    
    t1 = dt.datetime.now()
        
    #Print model report:
    print("\nModel Report")
    print("\nBest n_estimators : " + str(cvresult.shape[0]))
    print("RMSE : %.4g" % mean_squared_error(Ytrain.values, dtrain_predictions))
    print('Training time: %i seconds' % (t1 - t0).seconds)

#### Step 1: Fix learning rate and number of estimators for tuning tree-based parameters

In [108]:
xgb1 = XGBRegressor(
 learning_rate =0.3,
 n_estimators=10000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 scale_pos_weight=1,
 nthread=-1,
 seed=27)

XGBmodelfit(xgb1, data, target_log)

[0]	train-rmse:4.22737+1.4e-05	test-rmse:4.22736+0.0002195
[10]	train-rmse:0.448065+0.000419	test-rmse:0.449236+0.0009345
[20]	train-rmse:0.414902+0.000226	test-rmse:0.417081+0.0006475
[30]	train-rmse:0.408299+0.000398	test-rmse:0.41152+6.85e-05
[40]	train-rmse:0.404025+0.000993	test-rmse:0.40815+0.0003455
[50]	train-rmse:0.400341+0.000996	test-rmse:0.405595+0.0005045
[60]	train-rmse:0.397861+0.0008125	test-rmse:0.4042+0.0006805
[70]	train-rmse:0.395646+0.001084	test-rmse:0.403033+0.0009105
[80]	train-rmse:0.393298+0.0004945	test-rmse:0.401486+0.0002455
[90]	train-rmse:0.391382+0.0004045	test-rmse:0.400841+0.0002245
[100]	train-rmse:0.389838+0.00046	test-rmse:0.400544+0.0002115
[110]	train-rmse:0.388278+0.0005205	test-rmse:0.399861+0.0003195
[120]	train-rmse:0.386961+0.0007305	test-rmse:0.39954+0.000405
[130]	train-rmse:0.385317+0.0007045	test-rmse:0.39885+0.000351
[140]	train-rmse:0.383711+0.0006935	test-rmse:0.398363+0.0004185
[150]	train-rmse:0.382451+0.0009075	test-rmse:0.398124+0.

#### Step 2: Tune max_depth and min_child_weight

In [112]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

xgb1 = XGBRegressor(
 learning_rate =0.3,
 n_estimators=665,
 #max_depth=5,
 #min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 seed=27)

gsearch1 = GridSearchCV(xgb1 , param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=-1,iid=False, cv=2, verbose=20)

#Fit
gsearch1.fit(data, target_log)

#Print scores
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 25.6min
[Parallel(n_jobs=-1)]: Done  11 out of  24 | elapsed: 28.7min remaining: 33.9min
[Parallel(n_jobs=-1)]: Done  13 out of  24 | elapsed: 41.1min remaining: 34.8min
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed: 44.5min remaining: 26.7min
[Parallel(n_jobs=-1)]: Done  17 out of  24 | elapsed: 52.4min remaining: 21.6min
[Parallel(n_jobs=-1)]: Done  19 out of  24 | elapsed: 63.1min remaining: 16.6min
[Parallel(n_jobs=-1)]: Done  21 out o

([mean: -0.16060, std: 0.00012, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: -0.16080, std: 0.00041, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: -0.16076, std: 0.00007, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: -0.15540, std: 0.00027, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: -0.15551, std: 0.00002, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: -0.15535, std: 0.00020, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: -0.15503, std: 0.00008, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: -0.15453, std: 0.00005, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: -0.15494, std: 0.00041, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: -0.15749, std: 0.00007, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: -0.15803, std: 0.00052, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: -0.15789, std: 0.00002, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 7, 'min_chil

In [113]:
XGBmodelfit(gsearch1.best_estimator_, data, target_log)

[0]	train-rmse:4.22629+4.5e-05	test-rmse:4.22636+0.0002265
[10]	train-rmse:0.430665+0.0006635	test-rmse:0.43444+0.001239
[20]	train-rmse:0.398298+0.000124	test-rmse:0.406546+0.000459
[30]	train-rmse:0.39004+0.000779	test-rmse:0.401718+0.0002185
[40]	train-rmse:0.383791+0.001006	test-rmse:0.398533+0.000118
[50]	train-rmse:0.379731+0.0014515	test-rmse:0.397421+0.0001825
[60]	train-rmse:0.376082+0.0015195	test-rmse:0.396543+0.0004905
[70]	train-rmse:0.372679+0.001158	test-rmse:0.39586+0.000224
[80]	train-rmse:0.369534+0.0012065	test-rmse:0.395027+0.0001165
[90]	train-rmse:0.367053+0.000715	test-rmse:0.394624+0.000178
[100]	train-rmse:0.364228+0.0002015	test-rmse:0.394314+0.000564
[110]	train-rmse:0.361997+0.000426	test-rmse:0.394221+0.000201
[120]	train-rmse:0.359851+0.0004415	test-rmse:0.394006+0.0003505
[130]	train-rmse:0.357794+0.000168	test-rmse:0.393686+0.0005285
[140]	train-rmse:0.355931+0.000256	test-rmse:0.39342+0.0004945
[150]	train-rmse:0.353912+0.00018	test-rmse:0.393379+0.0005

#### Step 3: Tune gamma

In [5]:
param_test1 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

xgb1 = XGBRegressor(
 learning_rate =0.3,
 n_estimators=665,
 max_depth=7,
 min_child_weight=3,
 #gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 seed=27)

gsearch1 = GridSearchCV(xgb1 , param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=-1,iid=False, cv=2, verbose=20)

#Fit
gsearch1.fit(data, target_log)

#Print scores
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 23.6min
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed: 24.9min remaining: 99.6min
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed: 25.7min remaining: 59.9min
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed: 25.8min remaining: 38.6min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 26.3min remaining: 26.3min
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed: 26.9min remaining: 17.9min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed: 27.2min remaining: 11.7min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed: 27.5min remaining:  6.9min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 32.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 32.9min finished
C:\Users\Clement\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. Th

([mean: -0.15457, std: 0.00001, params: {'gamma': 0.0},
  mean: -0.15500, std: 0.00002, params: {'gamma': 0.1},
  mean: -0.15476, std: 0.00055, params: {'gamma': 0.2},
  mean: -0.15469, std: 0.00016, params: {'gamma': 0.3},
  mean: -0.15452, std: 0.00010, params: {'gamma': 0.4}],
 {'gamma': 0.4},
 -0.15451592362663652)

#### Step 4: Tune subsample and colsample_bytree

In [8]:
param_test1 = {
 'subsample':[i/10.0 for i in range(9,13)],
 'colsample_bytree':[i/10.0 for i in range(9,13)]
}

xgb1 = XGBRegressor(
 learning_rate =0.3,
 n_estimators=665,
 max_depth=7,
 min_child_weight=3,
 gamma=0.4,
 #subsample=0.8,
 #colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 seed=27)

gsearch1 = GridSearchCV(xgb1 , param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=-1,iid=False, cv=2, verbose=20)

#Fit
gsearch1.fit(data, target_log)

#Print scores
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 26.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 27.9min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 28.0min


JoblibXGBoostError: JoblibXGBoostError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Clement\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Clement\Anaconda3\lib\

#### Step 5: Tuning Regularization Parameters

In [10]:
param_test1 = {
 #'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100,1000,10000]
 'reg_alpha':[100,1000,10000]
}

xgb1 = XGBRegressor(
 learning_rate =0.3,
 n_estimators=665,
 max_depth=7,
 min_child_weight=3,
 gamma=0.4,
 subsample=0.9,
 colsample_bytree=0.9,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 seed=27)

gsearch1 = GridSearchCV(xgb1 , param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=-1,iid=False, cv=2, verbose=20)

#Fit
gsearch1.fit(data, target_log)

#Print scores
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  3.7min remaining:  7.5min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  4.7min remaining:  4.7min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  4.9min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  9.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  9.2min finished
C:\Users\Clement\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.14677, std: 0.00007, params: {'reg_alpha': 100},
  mean: -0.16942, std: 0.00050, params: {'reg_alpha': 1000},
  mean: -0.20881, std: 0.00059, params: {'reg_alpha': 10000}],
 {'reg_alpha': 100},
 -0.14676679858422431)

#### Step 6: Reducing Learning Rate

In [15]:
xgb1 = XGBRegressor(
 learning_rate =0.1,
 n_estimators=50000,
 max_depth=7,
 min_child_weight=3,
 gamma=0.4,
 subsample=0.9,
 colsample_bytree=0.9,
 objective= 'reg:linear',
 nthread=-1,
 scale_pos_weight=1,
 reg_alpha=100,
 seed=27)

XGBmodelfit(xgb1, data, target_log)

[0]	train-rmse:5.42211+0.000123	test-rmse:5.42213+0.000221
[10]	train-rmse:1.94064+3.55e-05	test-rmse:1.94091+0.0003735
[20]	train-rmse:0.793302+5.5e-05	test-rmse:0.794159+0.000618
[30]	train-rmse:0.486876+0.0001745	test-rmse:0.488603+0.000548
[40]	train-rmse:0.426585+0.0004545	test-rmse:0.428839+0.0002505
[50]	train-rmse:0.413461+0.0002085	test-rmse:0.416029+0.000446
[60]	train-rmse:0.408566+0.0001985	test-rmse:0.411394+0.000426
[70]	train-rmse:0.405382+0.000194	test-rmse:0.408431+0.000439
[80]	train-rmse:0.402895+0.0001595	test-rmse:0.406154+0.0004565
[90]	train-rmse:0.401059+0.000102	test-rmse:0.404479+0.000495
[100]	train-rmse:0.399592+3.2e-05	test-rmse:0.403174+0.000557
[110]	train-rmse:0.398076+0.0001225	test-rmse:0.401797+0.0004715
[120]	train-rmse:0.397039+0.0001735	test-rmse:0.400892+0.0004135
[130]	train-rmse:0.395833+0.00028	test-rmse:0.399831+0.000307
[140]	train-rmse:0.394633+0.0004485	test-rmse:0.398764+0.0001705
[150]	train-rmse:0.393508+0.0005385	test-rmse:0.397787+7.2e

[1290]	train-rmse:0.372379+0.000668	test-rmse:0.38229+0.000275
[1300]	train-rmse:0.372328+0.000672	test-rmse:0.382265+0.000276
[1310]	train-rmse:0.372293+0.000671	test-rmse:0.382252+0.000277
[1320]	train-rmse:0.372242+0.000671	test-rmse:0.382229+0.000284
[1330]	train-rmse:0.372188+0.00067	test-rmse:0.382203+0.0002885
[1340]	train-rmse:0.372148+0.0006645	test-rmse:0.382181+0.000291
[1350]	train-rmse:0.372115+0.0006665	test-rmse:0.382167+0.000292
[1360]	train-rmse:0.37208+0.0006705	test-rmse:0.382149+0.0002855
[1370]	train-rmse:0.37203+0.00067	test-rmse:0.382125+0.000291
[1380]	train-rmse:0.371997+0.0006695	test-rmse:0.382108+0.0002965
[1390]	train-rmse:0.371961+0.0006685	test-rmse:0.382092+0.0002995
[1400]	train-rmse:0.371918+0.0006695	test-rmse:0.382068+0.000303
[1410]	train-rmse:0.371885+0.000669	test-rmse:0.382053+0.000305
[1420]	train-rmse:0.371845+0.0006615	test-rmse:0.382033+0.000309
[1430]	train-rmse:0.371801+0.000673	test-rmse:0.382008+0.0003035
[1440]	train-rmse:0.371771+0.0006

[2560]	train-rmse:0.369071+0.000744	test-rmse:0.380825+0.000312
[2570]	train-rmse:0.369049+0.0007485	test-rmse:0.380814+0.000307
[2580]	train-rmse:0.369032+0.0007545	test-rmse:0.38081+0.0003065
[2590]	train-rmse:0.36901+0.000757	test-rmse:0.380802+0.000311
[2600]	train-rmse:0.368992+0.0007545	test-rmse:0.380796+0.0003125
[2610]	train-rmse:0.368977+0.0007535	test-rmse:0.380787+0.000313
[2620]	train-rmse:0.368962+0.000756	test-rmse:0.380784+0.00031
[2630]	train-rmse:0.368942+0.000751	test-rmse:0.380776+0.000312
[2640]	train-rmse:0.368926+0.0007505	test-rmse:0.380769+0.0003135
[2650]	train-rmse:0.368905+0.000757	test-rmse:0.380761+0.0003085
[2660]	train-rmse:0.368885+0.0007535	test-rmse:0.380754+0.0003095
[2670]	train-rmse:0.368877+0.0007565	test-rmse:0.380754+0.0003085
[2680]	train-rmse:0.368865+0.0007575	test-rmse:0.380749+0.0003055
[2690]	train-rmse:0.368849+0.0007525	test-rmse:0.380743+0.000308
[2700]	train-rmse:0.368829+0.000749	test-rmse:0.380733+0.0003105
[2710]	train-rmse:0.368813

[3830]	train-rmse:0.367436+0.00076	test-rmse:0.380224+0.000326
[3840]	train-rmse:0.367427+0.000759	test-rmse:0.380221+0.000327
[3850]	train-rmse:0.367419+0.00076	test-rmse:0.380219+0.000325
[3860]	train-rmse:0.367407+0.000754	test-rmse:0.380216+0.000328
[3870]	train-rmse:0.367395+0.0007545	test-rmse:0.380215+0.000326
[3880]	train-rmse:0.367388+0.000753	test-rmse:0.380212+0.0003275
[3890]	train-rmse:0.367376+0.000753	test-rmse:0.380207+0.0003285
[3900]	train-rmse:0.367371+0.000753	test-rmse:0.380205+0.000328
[3910]	train-rmse:0.367356+0.0007525	test-rmse:0.380199+0.000327
[3920]	train-rmse:0.367343+0.0007515	test-rmse:0.380193+0.0003275
[3930]	train-rmse:0.367335+0.0007535	test-rmse:0.380188+0.0003265
[3940]	train-rmse:0.367325+0.000753	test-rmse:0.380185+0.0003275
[3950]	train-rmse:0.367313+0.000748	test-rmse:0.380179+0.00033
[3960]	train-rmse:0.367306+0.0007515	test-rmse:0.380177+0.0003285
[3970]	train-rmse:0.367293+0.0007555	test-rmse:0.380172+0.000327
[3980]	train-rmse:0.367279+0.00

[5100]	train-rmse:0.366377+0.000771	test-rmse:0.379877+0.0003225
[5110]	train-rmse:0.366369+0.000773	test-rmse:0.379875+0.0003215
[5120]	train-rmse:0.36636+0.000774	test-rmse:0.379872+0.0003205
[5130]	train-rmse:0.366353+0.0007715	test-rmse:0.379871+0.00032
[5140]	train-rmse:0.366347+0.000772	test-rmse:0.379869+0.000322
[5150]	train-rmse:0.366335+0.0007725	test-rmse:0.379866+0.0003205
[5160]	train-rmse:0.366325+0.000774	test-rmse:0.379861+0.0003215
[5170]	train-rmse:0.366317+0.000771	test-rmse:0.379857+0.000322
[5180]	train-rmse:0.366313+0.000771	test-rmse:0.379855+0.000323
[5190]	train-rmse:0.366304+0.000775	test-rmse:0.379853+0.0003235
[5200]	train-rmse:0.366293+0.0007745	test-rmse:0.379848+0.000323
[5210]	train-rmse:0.366289+0.0007715	test-rmse:0.379848+0.0003235
[5220]	train-rmse:0.366277+0.000773	test-rmse:0.379846+0.0003255
[5230]	train-rmse:0.366274+0.0007755	test-rmse:0.379846+0.000325
[5240]	train-rmse:0.366266+0.0007765	test-rmse:0.379845+0.0003265
[5250]	train-rmse:0.36626+0

...

[7640]	train-rmse:0.364928+0.0007565	test-rmse:0.379464+0.000361
[7650]	train-rmse:0.364925+0.000758	test-rmse:0.379463+0.00036
[7660]	train-rmse:0.364921+0.00076	test-rmse:0.37946+0.0003585
[7670]	train-rmse:0.364919+0.0007595	test-rmse:0.37946+0.0003585
[7680]	train-rmse:0.364916+0.000758	test-rmse:0.37946+0.000358
[7690]	train-rmse:0.364912+0.000757	test-rmse:0.37946+0.0003585
[7700]	train-rmse:0.364907+0.000758	test-rmse:0.379458+0.000358
[7710]	train-rmse:0.364903+0.000755	test-rmse:0.379456+0.0003595
[7720]	train-rmse:0.364899+0.0007545	test-rmse:0.379456+0.0003595
[7730]	train-rmse:0.364893+0.000756	test-rmse:0.379453+0.0003615
[7740]	train-rmse:0.364888+0.0007555	test-rmse:0.379453+0.000361
[7750]	train-rmse:0.364884+0.000754	test-rmse:0.379452+0.000362
[7760]	train-rmse:0.364877+0.0007545	test-rmse:0.379451+0.000365
[7770]	train-rmse:0.364874+0.000753	test-rmse:0.37945+0.000366
[7780]	train-rmse:0.364871+0.000752	test-rmse:0.379448+0.0003675
[7790]	train-rmse:0.364868+0.000753

[8910]	train-rmse:0.364395+0.0007865	test-rmse:0.379335+0.0003635
[8920]	train-rmse:0.364393+0.000787	test-rmse:0.379335+0.0003635
[8930]	train-rmse:0.36439+0.000788	test-rmse:0.379333+0.0003625
[8940]	train-rmse:0.364388+0.000789	test-rmse:0.379333+0.000362
[8950]	train-rmse:0.364384+0.000789	test-rmse:0.379331+0.000363
[8960]	train-rmse:0.36438+0.000786	test-rmse:0.37933+0.000364
[8970]	train-rmse:0.364373+0.000788	test-rmse:0.379329+0.000363
[8980]	train-rmse:0.36437+0.0007875	test-rmse:0.379329+0.000362
[8990]	train-rmse:0.364366+0.0007885	test-rmse:0.379328+0.000361
[9000]	train-rmse:0.364359+0.0007885	test-rmse:0.379326+0.000363
[9010]	train-rmse:0.364353+0.0007875	test-rmse:0.379326+0.000365
[9020]	train-rmse:0.364348+0.0007885	test-rmse:0.379327+0.000364
[9030]	train-rmse:0.364344+0.0007905	test-rmse:0.379325+0.0003625
[9040]	train-rmse:0.364339+0.000792	test-rmse:0.379324+0.000363
[9050]	train-rmse:0.364335+0.0007935	test-rmse:0.379324+0.0003625
[9060]	train-rmse:0.364331+0.00

[10180]	train-rmse:0.363943+0.000808	test-rmse:0.379209+0.0003715
[10190]	train-rmse:0.363939+0.000808	test-rmse:0.379209+0.000372
[10200]	train-rmse:0.363937+0.000809	test-rmse:0.379208+0.000371
[10210]	train-rmse:0.363934+0.000807	test-rmse:0.379208+0.00037
[10220]	train-rmse:0.363931+0.000809	test-rmse:0.379208+0.00037
[10230]	train-rmse:0.36393+0.00081	test-rmse:0.379209+0.0003705
[10240]	train-rmse:0.363925+0.000813	test-rmse:0.379207+0.000369
[10250]	train-rmse:0.363924+0.000812	test-rmse:0.379207+0.000369
[10260]	train-rmse:0.363922+0.0008125	test-rmse:0.379207+0.0003685
[10270]	train-rmse:0.363919+0.000814	test-rmse:0.379207+0.000368
[10280]	train-rmse:0.363916+0.0008165	test-rmse:0.379206+0.000368
[10290]	train-rmse:0.363912+0.000815	test-rmse:0.379204+0.0003685
[10300]	train-rmse:0.363911+0.0008155	test-rmse:0.379204+0.0003685
[10310]	train-rmse:0.363907+0.000813	test-rmse:0.379205+0.000368
[10320]	train-rmse:0.363903+0.0008145	test-rmse:0.379203+0.000366
[10330]	train-rmse:0

[11430]	train-rmse:0.36355+0.000815	test-rmse:0.379121+0.0003745
[11440]	train-rmse:0.363548+0.0008145	test-rmse:0.379121+0.000375
[11450]	train-rmse:0.363544+0.000816	test-rmse:0.37912+0.0003725
[11460]	train-rmse:0.36354+0.000819	test-rmse:0.37912+0.000372


KeyboardInterrupt: 

In [ ]:
xgb1.n_estimators